<a href="https://colab.research.google.com/github/graviraja/100-Days-of-NLP/blob/embeddings/embeddings/ELMo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ELMo Embeddings

ELMo is a deep contextualized word representation that models:
- complex characteristics of word use (e.g., syntax and semantics)
-  how these uses vary across linguistic contexts (i.e., to model polysemy).

These word vectors are learned functions of the internal states of a deep bidirectional language model (biLM), which is pre-trained on a large text corpus.



![elmo arch](https://drive.google.com/uc?id=19zI1rtdTW4INZ4PNAcyYeWlvl7UMcysN)

## Resources

- [Allennlp](https://allennlp.org/elmo)
- [Analytics Vidhya Post on ELMo](https://www.analyticsvidhya.com/blog/2019/03/learn-to-use-elmo-to-extract-features-from-text/?utm_source=blog&utm_medium=pretrained-word-embeddings-nlp)

- [ELMo talk](https://www.youtube.com/watch?v=9JfGxKkmBc0)

- [NLP with Elmo and Flair](https://www.youtube.com/watch?v=ZEhWpZGlJvE)

- [ELMo Paper](https://arxiv.org/pdf/1802.05365.pdf)

- [Img ref](http://www.realworldnlpbook.com/blog/improving-sentiment-analyzer-using-elmo.html)

## Difference between ELMo and Word2Vec/GloVe Embeddings

### ELMo
---
**Advantages:**

- Embeddings are context dependent
    - For instance, for the same example above “He went to the prison cell with his cell phone to extract blood cell samples from inmates”, both Elmo and BERT would generate different vectors for the three vectors for cell. The first cell (prison cell case) , for instance would be closer to words like incarceration, crime etc. whereas the second “cell” (phone case) would be closer to words like iphone, android, galaxy etc..

**Disadvantages:**

- NO Arithmetic properties
- NO Cosine similarities

### Word2Vec / GloVe
---
**Advantages:**

- Arithmetic Properties (king - man + women = queen)
- Cosine Similarities (dog ~= husky)

**Disadvantages:**

- Embeddings are **not** context dependent
    - That is the one numeric representation of a word (which we call embedding/vector) regardless of where the words occurs in a sentence and regardless of the different meanings they may have. For instance, after we train word2vec/Glove on a corpus (unsupervised training - no labels needed) we get as output one vector representation for, say the word “cell”. So even if we had a sentence like “He went to the prison cell with his cell phone to extract blood cell samples from inmates”, where the word cell has different meanings based on the sentence context, these models just collapse them all into one vector for “cell” in their output.


---

[source](https://www.quora.com/What-are-the-main-differences-between-the-word-embeddings-of-ELMo-BERT-Word2vec-and-GloVe)



# AllenNLP Library

[AllenNLP]((https://allennlp.org/)) is:

- #### Deeplearning for NLP

  >  AllenNLP makes it easy to design and evaluate new deep learning models for nearly any NLP problem, along with the infrastructure to easily run them in the cloud or on your laptop.

- #### State of the art models

  >  AllenNLP includes reference implementations of high quality models for both core NLP problems (e.g. semantic role labeling) and NLP applications (e.g. textual entailment).



## Initial Setup

In [0]:
!pip install allennlp

## Using Allennlp for ELMo Embeddings

In [0]:
import torch
from allennlp.commands.elmo import ElmoEmbedder

In [0]:
# small model
options_file = "https://s3-us-west-2.amazonaws.com/allennlp/models/elmo/2x1024_128_2048cnn_1xhighway/elmo_2x1024_128_2048cnn_1xhighway_options.json"
weight_file = "https://s3-us-west-2.amazonaws.com/allennlp/models/elmo/2x1024_128_2048cnn_1xhighway/elmo_2x1024_128_2048cnn_1xhighway_weights.hdf5"

# other models option, weight files are available here: https://allennlp.org/elmo

elmo = ElmoEmbedder(options_file=options_file, weight_file=weight_file)

In [0]:
sentence = "I ate an apple for breakfast"
words = sentence.split()

In [0]:
word_embeds = elmo.embed_sentence(words)

In [45]:
print(word_embeds.shape)

(3, 6, 256)


As we can in the architecture diagram, embeddings are created by combing the representations from `input` layer, `first layer` and `final layer`. But ELMo word embeddings can be constructed by combining ELMo layers in different ways. The available combination strategies are:

**`all`**: Use the concatenation of the three ELMo layers.

**`top`**: Use the top ELMo layer.

**`average`**: Use the average of the three ELMo layers.


As we can see from above the `word_embeds` is of shape `(3, 6, 256)` indicating `(layers, seq_length, emb_size)`

Let's see how can do the three operations mentioned above with the `word_embeds`



#### **`all`**: Use the concatenation of the three ELMo layers.

In [59]:
# convertion to torch tensors
word_tensors = torch.tensor(word_embeds)
word_tensors.shape

torch.Size([3, 6, 256])

In [61]:
all_mode = lambda x: torch.cat(x, 0)

all_embs = []
for token, token_idx in zip(words, range(len(words))):
    # get the embeddings from each layer
    elmo_embedding_layers = [
        torch.FloatTensor(word_tensors[0, token_idx, :]),
        torch.FloatTensor(word_tensors[1, token_idx, :]),
        torch.FloatTensor(word_tensors[2, token_idx, :])
    ]
    # concatenate all the layer embeddings
    word_embedding = all_mode(elmo_embedding_layers)
    all_embs.append(word_embedding)

len(all_embs)

6

In [62]:
# upon concatenating all the three layers the shape of embedding would be: 256 + 256 + 256 = 768
all_embs[0].shape

torch.Size([768])

#### **`top`**: Use the top ELMo layer

In [64]:
top_mode = lambda x: x[-1]

top_embs = []
for token, token_idx in zip(words, range(len(words))):
    # get the embeddings from each layer
    elmo_embedding_layers = [
        torch.FloatTensor(word_tensors[0, token_idx, :]),
        torch.FloatTensor(word_tensors[1, token_idx, :]),
        torch.FloatTensor(word_tensors[2, token_idx, :])
    ]
    # get only the top embedding from all the layers
    word_embedding = top_mode(elmo_embedding_layers)
    top_embs.append(word_embedding)

len(top_embs)

6

In [65]:
# upon getting the embedding from top layer the shape of embedding would be: 256
top_embs[0].shape

torch.Size([256])

#### **`average`**: Use the average of the three ELMo layers

In [67]:
average_mode = lambda x: torch.mean(torch.stack(x), 0)

avg_embs = []
for token, token_idx in zip(words, range(len(words))):
    # get the embeddings from each layer
    elmo_embedding_layers = [
        torch.FloatTensor(word_tensors[0, token_idx, :]),
        torch.FloatTensor(word_tensors[1, token_idx, :]),
        torch.FloatTensor(word_tensors[2, token_idx, :])
    ]
    # get average of all the embeddings from all the layers
    word_embedding = average_mode(elmo_embedding_layers)
    avg_embs.append(word_embedding)

len(avg_embs)

6

In [69]:
# upon averaging the embedding from all layers the shape of embedding would be: (256 + 256 + 256) / 3 = 256
avg_embs[0].shape

torch.Size([256])

# Flair Library

[Flair](https://github.com/flairNLP/flair) is:


> A powerful NLP library. Flair allows you to apply our state-of-the-art natural language processing (NLP) models to your text, such as named entity recognition (NER), part-of-speech tagging (PoS), sense disambiguation and classification.

> Multilingual. Thanks to the Flair community, we support a rapidly growing number of languages. We also now include 'one model, many languages' taggers, i.e. single models that predict PoS or NER tags for input text in various languages.

> A text embedding library. Flair has simple interfaces that allow you to use and combine different word and document embeddings, including our proposed Flair embeddings, BERT embeddings and ELMo embeddings.

> A PyTorch NLP framework. Our framework builds directly on PyTorch, making it easy to train your own models and experiment with new approaches using Flair embeddings and classes.


## Initial Setup

In [1]:
!pip install flair

     |████████████████████████████████| 143kB 3.5MB/s 
     |████████████████████████████████| 798kB 9.1MB/s 
     |████████████████████████████████| 983kB 18.3MB/s 
     |████████████████████████████████| 645kB 24.5MB/s 
     |████████████████████████████████| 256kB 31.1MB/s 
     |████████████████████████████████| 1.1MB 41.8MB/s 
     |████████████████████████████████| 890kB 23.5MB/s 
     |████████████████████████████████| 3.8MB 43.7MB/s 
  Created wheel for sqlitedict: filename=sqlitedict-1.6.0-cp36-none-any.whl size=14689 sha256=11a7cc29994d4d4e091adcdb082813f463aae8de7280fb6427a26bfb74147a82
  Stored in directory: /root/.cache/pip/wheels/bd/57/d3/907c3ee02d35e66f674ad0106e61f06eeeb98f6ee66a6cc3fe
  Created wheel for mpld3: filename=mpld3-0.3-cp36-none-any.whl size=116679 sha256=e0c990d954f3d44f5a4a48f83b1806b8506f2433aa7269cf00742919b52145e1
  Stored in directory: /root/.cache/pip/wheels/c0/47/fb/8a64f89aecfe0059830479308ad42d62e898a3e3cefdf6ba28
  Created wheel for segtok: filen

In [5]:
!pip install allennlp

     |████████████████████████████████| 7.6MB 3.1MB/s 
     |████████████████████████████████| 5.2MB 39.7MB/s 
     |████████████████████████████████| 204kB 35.6MB/s 
     |████████████████████████████████| 30.9MB 139kB/s 
     |████████████████████████████████| 245kB 45.0MB/s 
     |████████████████████████████████| 163kB 43.7MB/s 
     |████████████████████████████████| 256kB 47.3MB/s 
     |████████████████████████████████| 61kB 6.7MB/s 
     |████████████████████████████████| 51kB 4.7MB/s 
     |████████████████████████████████| 133kB 43.3MB/s 
     |████████████████████████████████| 51kB 5.5MB/s 
     |████████████████████████████████| 92kB 8.1MB/s 
     |████████████████████████████████| 3.2MB 44.8MB/s 
     |████████████████████████████████| 2.1MB 41.3MB/s 
  Created wheel for numpydoc: filename=numpydoc-0.9.2-cp36-none-any.whl size=31893 sha256=b9535b686e26d7451d46d52bd9675ba0052f92b85e2029c9e2b50da8e9218cb9
  Stored in directory: /root/.cache/pip/wheels/96/f3/52/25c8e1f4063766

## Using Flair for ELMo Embeddings

Resources:

- [Flair tutorial on Word Embeddings](https://github.com/flairNLP/flair/blob/master/resources/docs/TUTORIAL_3_WORD_EMBEDDING.md)

- [List of available embeddings in Flair](https://github.com/flairNLP/flair/blob/master/resources/docs/TUTORIAL_4_ELMO_BERT_FLAIR_EMBEDDING.md)

In [0]:
from flair.data import Sentence
from flair.embeddings import ELMoEmbeddings

In [0]:
# by default the mode is all
embedding = ELMoEmbeddings(model="small")

In [0]:
sentence = Sentence('I ate an apple for breakfast')

In [74]:
embedding.embed(sentence)

[Sentence: "I ate an apple for breakfast" - 6 Tokens]

In [75]:
for token in sentence:
    print(f"{token} \t emb: {token.embedding.shape}")

Token: 1 I 	 emb: torch.Size([768])
Token: 2 ate 	 emb: torch.Size([768])
Token: 3 an 	 emb: torch.Size([768])
Token: 4 apple 	 emb: torch.Size([768])
Token: 5 for 	 emb: torch.Size([768])
Token: 6 breakfast 	 emb: torch.Size([768])


#### **`all`**: Use the concatenation of the three ELMo layers.

In [76]:
# upon concatenating all the three layers the shape of embedding would be: 256 + 256 + 256 = 768
for token in sentence:
    print(f"{token} \t emb: {token.embedding.shape}")

Token: 1 I 	 emb: torch.Size([768])
Token: 2 ate 	 emb: torch.Size([768])
Token: 3 an 	 emb: torch.Size([768])
Token: 4 apple 	 emb: torch.Size([768])
Token: 5 for 	 emb: torch.Size([768])
Token: 6 breakfast 	 emb: torch.Size([768])


#### Notes:

- **`top`** and **`average`** modes are not supported in flair==0.4.5.

- There has been PR raised for this [here](https://github.com/flairNLP/flair/pull/1547)

- In future release the support will be there by providing an extra argument `embedding_type`
    - [ELMo blog by Flair](https://github.com/flairNLP/flair/blob/master/resources/docs/embeddings/ELMO_EMBEDDINGS.md)
    - [Source code](https://github.com/flairNLP/flair/blob/master/flair/embeddings/token.py#L1543)
